In [3]:
pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 4.9 MB/s 
     |████████████████████████████████| 578.0 MB 19 kB/s 
     |████████████████████████████████| 438 kB 29.3 MB/s 
     |████████████████████████████████| 1.7 MB 33.8 MB/s 
     |████████████████████████████████| 5.9 MB 31.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

In [1]:
from google.colab import files
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
import matplotlib as mt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input,Dropout
from keras.models import Model

In [2]:
uploaded=files.upload()

Saving antisemitism_combined.csv to antisemitism_combined.csv


In [3]:
df=pd.read_csv('/content/antisemitism_combined.csv')

In [6]:
df['classification'].value_counts()

0    877
1    344
Name: classification, dtype: int64

In [9]:
label0=df[df['classification']==0]
label1=df[df['classification']==1]
newlabel0=label0.sample(len(label1))

In [14]:
new_df=pd.concat([newlabel0,label1])


In [15]:
df_merged=df[['classification','text']]


In [40]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
y=lb.fit_transform(df_merged['classification'])
y

array([0, 1, 1, ..., 0, 0, 1])

In [53]:
X_train,X_test,Y_train,Y_test=train_test_split(df_merged['text'],y,stratify=y)
Y_train = np.asarray(Y_train).astype(np.float32)
X_train = np.array([np.array(val) for val in X_train])

In [16]:
preprocess_url=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder_url=hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [10]:
def sentence_embedding( sentences):
  preprocesstext= preprocess_url(sentences)
  return encoder_url(preprocesstext)['pooled_output']

sentence_embedding(['Dhanush','apple'])


<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8499283 , -0.35527462, -0.13184536, ...,  0.13185869,
        -0.6033863 ,  0.7870267 ],
       [-0.81964606, -0.29609635,  0.20951606, ...,  0.25593287,
        -0.5874301 ,  0.84345585]], dtype=float32)>

In [37]:
e=sentence_embedding([
    'Not what you think Propagandist PRO zionist (Amos Oz supporter) Parents —&gt; Hollywood',
])
e1=sentence_embedding([
    "I'm probably blackpilled due to years of zog programming but I cant help but think that he watches other men please his wife."])


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]],[e1[0]])

array([[0.9331068]], dtype=float32)

In [32]:
input_layer=Input(shape=(),dtype=tf.string,name='text')
preprocess=preprocess_url(input_layer)
embedding=encoder_url(preprocess)

l1=Dropout(0.1,name='dropout')(embedding['pooled_output'])
l1=Dense(1,name='output',activation='sigmoid')(l1)

model=Model(inputs=[input_layer],outputs=[l1])


In [33]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [34]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='binaryaccuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name= 'recall')
]
model.compile(loss='categorical_crossentropy', 
                   optimizer='adam', 
                   metrics=METRICS)

In [54]:
model.fit(X_train, Y_train, epochs=1)

29/29 [==============================] - 505s 17s/step - loss: 0.0000e+00 - binaryaccuracy: 0.7049 - precision: 0.3000 - recall: 0.0349
